In [2]:
import pandas as pd
import requests as rq
import json
import time

In [5]:
response_raw = rq.get('https://fantasy.premierleague.com/api/element-summary/355/')
response: json = response_raw.json()

In [6]:
response

{'fixtures': [{'id': 16,
   'code': 2367553,
   'team_h': 13,
   'team_h_score': None,
   'team_a': 15,
   'team_a_score': None,
   'event': 2,
   'finished': False,
   'minutes': 0,
   'provisional_start_time': False,
   'kickoff_time': '2023-08-19T19:00:00Z',
   'event_name': 'Gameweek 2',
   'is_home': True,
   'difficulty': 4},
  {'id': 30,
   'code': 2367567,
   'team_h': 17,
   'team_h_score': None,
   'team_a': 13,
   'team_a_score': None,
   'event': 3,
   'finished': False,
   'minutes': 0,
   'provisional_start_time': False,
   'kickoff_time': '2023-08-27T13:00:00Z',
   'event_name': 'Gameweek 3',
   'is_home': False,
   'difficulty': 2},
  {'id': 39,
   'code': 2367576,
   'team_h': 13,
   'team_h_score': None,
   'team_a': 10,
   'team_a_score': None,
   'event': 4,
   'finished': False,
   'minutes': 0,
   'provisional_start_time': False,
   'kickoff_time': '2023-09-02T14:00:00Z',
   'event_name': 'Gameweek 4',
   'is_home': True,
   'difficulty': 2},
  {'id': 49,
   'code

In [130]:
d = []
entry = '21482'
entry_picks = [player['element'] for player in response['picks']]
d.append({entry: entry_picks})

In [133]:
d[0]['21482']

[275, 206, 506, 398, 19, 226, 353, 509, 396, 355, 60, 524, 326, 139, 405]

In [25]:
player_response_raw = rq.get('https://fantasy.premierleague.com/api/element-summary/355/')
player_response: json = player_response_raw.json()

In [26]:
player_response

{'fixtures': [{'id': 16,
   'code': 2367553,
   'team_h': 13,
   'team_h_score': None,
   'team_a': 15,
   'team_a_score': None,
   'event': 2,
   'finished': False,
   'minutes': 0,
   'provisional_start_time': False,
   'kickoff_time': '2023-08-19T19:00:00Z',
   'event_name': 'Gameweek 2',
   'is_home': True,
   'difficulty': 4},
  {'id': 30,
   'code': 2367567,
   'team_h': 17,
   'team_h_score': None,
   'team_a': 13,
   'team_a_score': None,
   'event': 3,
   'finished': False,
   'minutes': 0,
   'provisional_start_time': False,
   'kickoff_time': '2023-08-27T13:00:00Z',
   'event_name': 'Gameweek 3',
   'is_home': False,
   'difficulty': 2},
  {'id': 39,
   'code': 2367576,
   'team_h': 13,
   'team_h_score': None,
   'team_a': 10,
   'team_a_score': None,
   'event': 4,
   'finished': False,
   'minutes': 0,
   'provisional_start_time': False,
   'kickoff_time': '2023-09-02T14:00:00Z',
   'event_name': 'Gameweek 4',
   'is_home': True,
   'difficulty': 2},
  {'id': 49,
   'code

In [28]:
player_response['history'][0][]

{'element': 355,
 'fixture': 1,
 'opponent_team': 6,
 'total_points': 13,
 'was_home': False,
 'kickoff_time': '2023-08-11T19:00:00Z',
 'team_h_score': 0,
 'team_a_score': 3,
 'round': 1,
 'minutes': 79,
 'goals_scored': 2,
 'assists': 0,
 'clean_sheets': 1,
 'goals_conceded': 0,
 'own_goals': 0,
 'penalties_saved': 0,
 'penalties_missed': 0,
 'yellow_cards': 0,
 'red_cards': 0,
 'saves': 0,
 'bonus': 3,
 'bps': 56,
 'influence': '67.6',
 'creativity': '23.7',
 'threat': '48.0',
 'ict_index': '13.9',
 'starts': 1,
 'expected_goals': '0.60',
 'expected_assists': '0.05',
 'expected_goal_involvements': '0.65',
 'expected_goals_conceded': '0.23',
 'value': 140,
 'transfers_balance': 0,
 'selected': 7200159,
 'transfers_in': 0,
 'transfers_out': 0}

In [137]:
class Player:

    def __init__(self, element: int):
        try:
            self.player_history: list = []
            self.opponent_team: int = ''
            self.total_points: int = ''
            self.was_home: int = ''
            self.team_h_score: int = ''
            self.team_a_score: int = ''
            self.minutes: int = ''
            self.goals_scored: int = ''
            self.assists: int = ''
            self.clean_sheets: int = ''
            self.goals_conceded: int = ''
            self.own_goals: int = ''
            self.penalties_saved: int = ''
            self.penalties_missed: int = ''
            self.yellow_cards: int = ''
            self.red_cards: int = ''
            self.saves: int = ''
            self.bonus: int = ''
            self.bps: int = ''
            self.influence: float = ''
            self.creativity: float = ''
            self.threat: float = ''
            self.ict_index: float = ''
            self.starts: int = ''
            self.expected_goals: float = ''
            self.expected_assists: float = ''
            self.expected_goal_involvements: float = ''
            self.expected_goals_conceded: float = ''
            self.value: int = ''

            filtered_df = df_all[df_all['id_y'] == element]
            self.team_name = filtered_df['name'].iloc[0]
            self.first_name = filtered_df['first_name'].iloc[0]
            self.second_name = filtered_df['second_name'].iloc[0]
            self.position = position_dict[int(filtered_df['element_type'].iloc[0])]
            self.team_name = filtered_df['name'].iloc[0]
            self.team_name = filtered_df['name'].iloc[0]

            player_response_raw = rq.get(f'https://fantasy.premierleague.com/api/element-summary/{element}/')
            player_response: json = player_response_raw.json()

            for gw in player_response['history']:
                self.player_history.append(gw)

            gw_summary = player_response['history'][-1]

            self.opponent_team = int(gw_summary['opponent_team'])
            self.total_points = int(gw_summary['total_points'])
            self.was_home = int(gw_summary['was_home'])
            self.team_h_score = int(gw_summary['team_h_score'])
            self.team_a_score = int(gw_summary['team_a_score'])
            self.minutes = int(gw_summary['minutes'])
            self.goals_scored = int(gw_summary['goals_scored'])
            self.assists = int(gw_summary['assists'])
            self.clean_sheets = int(gw_summary['clean_sheets'])
            self.goals_conceded = int(player_response['history'][-1]['goals_conceded'])
            self.own_goals = int(player_response['history'][-1]['own_goals'])
            self.penalties_saved = int(player_response['history'][-1]['penalties_saved'])
            self.penalties_missed = int(player_response['history'][-1]['penalties_missed'])
            self.yellow_cards = int(player_response['history'][-1]['yellow_cards'])
            self.red_cards = int(player_response['history'][-1]['red_cards'])
            self.saves = int(player_response['history'][-1]['saves'])
            self.bonus = int(player_response['history'][-1]['bonus'])
            self.bps = int(player_response['history'][-1]['bps'])
            self.influence = float(player_response['history'][-1]['influence'])
            self.creativity = float(player_response['history'][-1]['creativity'])
            self.threat = float(player_response['history'][-1]['threat'])
            self.ict_index = float(player_response['history'][-1]['ict_index'])
            self.starts = int(player_response['history'][-1]['starts'])
            self.expected_goals = float(player_response['history'][-1]['expected_goals'])
            self.expected_assists = float(player_response['history'][-1]['expected_assists'])
            self.expected_goal_involvements = float(player_response['history'][-1]['expected_goal_involvements'])
            self.expected_goals_conceded = float(player_response['history'][-1]['expected_goals_conceded'])
            self.value = int(player_response['history'][-1]['value'])
            self.uniqueness = df_unique[df_unique['player'] == element]['unique'].iloc[0]
        except:
            pass


class Team:

    def __init__(self, entry: str):
        self.entry = entry
        self.df_unique = df_unique[df_unique['entry'] == entry]
        self.uniqueness = self.df_unique['unique'].mean()
        self.player_list: list = [p for p in self.df_unique['player']]
        self.player_dict: dict = {p: player_dict[p] for p in self.player_list}



url_bootstrap = 'https://fantasy.premierleague.com/api/bootstrap-static/'
bootstrap_raw = rq.get(url_bootstrap)
bootstrap: json = bootstrap_raw.json()

df_teams = pd.DataFrame(bootstrap['teams'])
df_players = pd.DataFrame(bootstrap['elements'])
df_all = pd.merge(df_teams, df_players, left_on='id', right_on='team')

position_dict: dict = {1: 'GOAL', 2: 'DEF', 3: 'MID', 4: 'FWD'}  # element_type

league_response_raw = rq.get('https://fantasy.premierleague.com/api/leagues-classic/27964/standings/')
league_response: json = league_response_raw.json()

participants_json = league_response['standings']['results']
participants_list: list = []

for p in league_response['standings']['results']:
    entry = p['entry']
    participant_response_raw = rq.get(f'https://fantasy.premierleague.com/api/entry/{entry}/event/1/picks/')
    participant_response: json = participant_response_raw.json()

    pick_dict = []
    entry_picks = [player['element'] for player in participant_response['picks']]

    participants_list.append({entry: entry_picks})
    time.sleep(0.5)


data = [(key, value) for team in participants_list for key, values in team.items() for value in values]

df_melted_players = pd.DataFrame(data, columns=['entry', 'player'])
df_groupby = df_melted_players.groupby('player').count().sort_values('entry', ascending=False)
df_groupby['unique'] = 1 - (df_groupby['entry'] - 1) / (len(participants_list) - 1)
df_unique = df.merge(df_groupby.drop('entry', axis=1), on='player', how='left')

player_dict: dict = {}

for p in df_groupby.reset_index()['player']:
    player_dict[p] = Player(p)

team_dict: dict = {}

print('hej')
for p in participants_list:
    team_dict[p] = Team(p)

print('hej')

hej


TypeError: unhashable type: 'dict'

In [125]:
df_melted_players = pd.DataFrame(data, columns=['entry', 'player'])
df_groupby = df_melted_players.groupby('player').count().sort_values('entry', ascending=False)
df_groupby['unique'] = 1 - (df_groupby['entry'] - 1) / (len(participants_list) - 1)
df_unique = df.merge(df_groupby.drop('entry', axis=1), on='player', how='left')

In [127]:
for p in df_groupby.reset_index()['player']:
    print(p)

355
19
396
131
368
5
14
405
415
135
60
524
398
373
195
473
597
17
143
308
226
29
430
290
108
51
402
20
433
291
263
132
12
28
353
509
113
109
349
538
501
206
313
6
346
303
352
275
197
304
148
354
211
26
134
526
326
53
293
8
33
499
343
139
316
119
447
504
365
507
112
106
500
602
140
490
599
403
49
43
209
228
230
13
533
391
427
37
424
36
419
466
428
429
502
491
496
577
572
552
541
540
539
532
528
15
523
513
506
407
31
497
412
192
406
252
128
302
282
278
145
271
258
256
245
48
243
152
220
207
160
205
198
167
306
307
126
309
401
397
55
382
172
58
63
361
358
356
72
101
103
350
344
336
324
377


In [123]:
df_unique

,entry,player,unique
0,1825965,275,0.928571
1,1825965,131,0.523810
2,1825965,20,0.857143
3,1825965,430,0.833333
4,1825965,497,1.000000
...,...,...,...
640,568537,433,0.857143
641,568537,145,1.000000
642,568537,206,0.904762
643,568537,397,1.000000


In [130]:
df_unique[df_unique['entry'] == 1825965]#['unique'].mean()

,entry,player,unique
0,1825965,275,0.928571
1,1825965,131,0.523810
2,1825965,20,0.857143
3,1825965,430,0.833333
4,1825965,497,1.000000
5,1825965,429,1.000000
6,1825965,14,0.619048
7,1825965,373,0.738095
8,1825965,15,1.000000
9,1825965,415,0.714286


In [133]:
[p for p in df_unique[df_unique['entry'] == 1825965]['player']]


[275, 131, 20, 430, 497, 429, 14, 373, 15, 415, 355, 291, 290, 539, 523]

In [72]:
url_bootstrap = 'https://fantasy.premierleague.com/api/bootstrap-static/'
bootstrap_raw = rq.get(url_bootstrap)

In [63]:
league_response_raw = rq.get('https://fantasy.premierleague.com/api/leagues-classic/27964/standings/')
league_response: json = league_response_raw.json()

In [65]:
league_response['standings']['results']

[{'id': 11957290,
  'event_total': 75,
  'player_name': 'Alvand Asgari',
  'rank': 1,
  'last_rank': 0,
  'rank_sort': 1,
  'total': 75,
  'entry': 1825965,
  'entry_name': 'ASCARI'},
 {'id': 9129900,
  'event_total': 70,
  'player_name': 'Albert Catana',
  'rank': 2,
  'last_rank': 0,
  'rank_sort': 2,
  'total': 70,
  'entry': 1416075,
  'entry_name': 'Point Snatchers FC'},
 {'id': 11961437,
  'event_total': 67,
  'player_name': 'Andreas Hedengrahn',
  'rank': 3,
  'last_rank': 0,
  'rank_sort': 3,
  'total': 67,
  'entry': 1826476,
  'entry_name': 'Djuuuu'},
 {'id': 3642937,
  'event_total': 67,
  'player_name': 'Viktor sjöberg',
  'rank': 3,
  'last_rank': 0,
  'rank_sort': 4,
  'total': 67,
  'entry': 587678,
  'entry_name': 'FC LAKEMOUNTAIN'},
 {'id': 35697940,
  'event_total': 67,
  'player_name': 'Richard Gjerde Norum',
  'rank': 3,
  'last_rank': 0,
  'rank_sort': 5,
  'total': 67,
  'entry': 2381195,
  'entry_name': 'Shaw Mee the Partey'},
 {'id': 15000997,
  'event_total': 6

In [146]:
df_teams

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1230,1285,1250,1250,1210,1320,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1115,1175,1130,1190,1100,1160,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1060,1095,1050,1100,1060,1090,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1125,1205,1120,1220,1130,1190,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1165,1210,1120,1200,1210,1240,131
5,90,0,None,6,0,Burnley,0,0,0,BUR,...,None,False,0,1060,1080,1060,1080,1060,1080,43
6,8,0,None,7,0,Chelsea,0,0,0,CHE,...,None,False,0,1115,1160,1130,1210,1100,1110,4
7,31,0,None,8,0,Crystal Palace,0,0,0,CRY,...,None,False,0,1100,1100,1140,1170,1080,1085,6
8,11,0,None,9,0,Everton,0,0,0,EVE,...,None,False,0,1075,1100,1070,1120,1080,1080,7
9,54,0,None,10,0,Fulham,0,0,0,FUL,...,None,False,0,1095,1100,1090,1090,1100,1140,34


In [62]:
df_all[df_all['id_y'] == 355]

,code_x,draw,form_x,id_x,loss,name,played,points,position,short_name,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
407,43,0,None,13,0,Man City,0,0,0,MCI,...,1,1,3,2,3,2,1,1,1.14,1.14


In [145]:
list_of_dicts = [
    {'name': 'Alice', 'age': 25, 'country': 'USA'},
    {'name': 'Bob', 'age': 30, 'country': 'Canada'},
    {'name': 'Charlie', 'age': 22, 'country': 'UK'}
]

for dictionary in list_of_dicts:
    keys = dictionary.keys()
    print("Keys:", keys)


Keys: dict_keys(['name', 'age', 'country'])
Keys: dict_keys(['name', 'age', 'country'])
Keys: dict_keys(['name', 'age', 'country'])
